In [1]:

import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, f1_score, accuracy_score,recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    mean_squared_error,
    mean_absolute_error,
    f1_score
)
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline

train = pd.read_csv(
    "../Dataset/Preparation_dir/final_data/feature_engineering_data/version2/train.csv"
)
test = pd.read_csv(
    "../Dataset/Preparation_dir/final_data/feature_engineering_data/version2/test.csv"
)

target = "act"

In [2]:

from sklearn.feature_selection import RFE

selected_features_list = [
    "BandAccX",
    "BandAccY",
    "BandAccZ",
    "speed",
    "rate",
    "BandAccX_diff",
    "BandAccY_diff",
    "BandAccZ_diff",
    "speed_diff",
    "rate_diff",
    "rateZone",
    "usage",
    "deviceStatus",
    "Acceleration_X",
    "Acceleration_Y",
    "Acceleration_Z",
    "MagneticField_X",
    "MagneticField_Y",
    "MagneticField_Z",
    "Orientation_X",
    "Orientation_Y",
    "Orientation_Z",
    "AngularVelocity_X",
    "AngularVelocity_Y",
    "AngularVelocity_Z",
    "Acceleration_X_diff",
    "Acceleration_Y_diff",
    "Acceleration_Z_diff",
    "MagneticField_X_diff",
    "MagneticField_Y_diff",
    "MagneticField_Z_diff",
    "Orientation_X_diff",
    "Orientation_Y_diff",
    "Orientation_Z_diff",
    "AngularVelocity_X_diff",
    "AngularVelocity_Y_diff",
    "AngularVelocity_Z_diff",
    "act",
    "Acceleration_X_rolling_min",
    "Acceleration_Y_rolling_min",
    "Acceleration_Z_rolling_min",
    "MagneticField_X_rolling_min",
    "MagneticField_Y_rolling_min",
    "MagneticField_Z_rolling_min",
    "Orientation_X_rolling_min",
    "Orientation_Y_rolling_min",
    "Orientation_Z_rolling_min",
    "AngularVelocity_X_rolling_min",
    "AngularVelocity_Y_rolling_min",
    "AngularVelocity_Z_rolling_min",
    "BandAccX_rolling_min",
    "BandAccY_rolling_min",
    "BandAccZ_rolling_min",
    "speed_rolling_min",
    "rate_rolling_min",
    "Acceleration_X_rolling_max",
    "Acceleration_Y_rolling_max",
    "Acceleration_Z_rolling_max",
    "MagneticField_X_rolling_max",
    "MagneticField_Y_rolling_max",
    "MagneticField_Z_rolling_max",
    "Orientation_X_rolling_max",
    "Orientation_Y_rolling_max",
    "Orientation_Z_rolling_max",
    "AngularVelocity_X_rolling_max",
    "AngularVelocity_Y_rolling_max",
    "AngularVelocity_Z_rolling_max",
    "BandAccX_rolling_max",
    "BandAccY_rolling_max",
    "BandAccZ_rolling_max",
    "speed_rolling_max",
    "rate_rolling_max",
    "Acceleration_X_rolling_std",
    "Acceleration_Y_rolling_std",
    "Acceleration_Z_rolling_std",
    "MagneticField_X_rolling_std",
    "MagneticField_Y_rolling_std",
    "MagneticField_Z_rolling_std",
    "Orientation_X_rolling_std",
    "Orientation_Y_rolling_std",
    "Orientation_Z_rolling_std",
    "AngularVelocity_X_rolling_std",
    "AngularVelocity_Y_rolling_std",
    "AngularVelocity_Z_rolling_std",
    "BandAccX_rolling_std",
    "BandAccY_rolling_std",
    "BandAccZ_rolling_std",
    "speed_rolling_std",
    "rate_rolling_std",
    "Acceleration_X_rolling_mean",
    "Acceleration_Y_rolling_mean",
    "Acceleration_Z_rolling_mean",
    "MagneticField_X_rolling_mean",
    "MagneticField_Y_rolling_mean",
    "MagneticField_Z_rolling_mean",
    "Orientation_X_rolling_mean",
    "Orientation_Y_rolling_mean",
    "Orientation_Z_rolling_mean",
    "AngularVelocity_X_rolling_mean",
    "AngularVelocity_Y_rolling_mean",
    "AngularVelocity_Z_rolling_mean",
    "BandAccX_rolling_mean",
    "BandAccY_rolling_mean",
    "BandAccZ_rolling_mean",
    "speed_rolling_mean",
    "rate_rolling_mean",
]

# 只选择selected_features_list中的特征
X_train = train[selected_features_list]
X_test = test[selected_features_list]

In [3]:
X_train.columns

Index(['BandAccX', 'BandAccY', 'BandAccZ', 'speed', 'rate', 'BandAccX_diff',
       'BandAccY_diff', 'BandAccZ_diff', 'speed_diff', 'rate_diff',
       ...
       'Orientation_Y_rolling_mean', 'Orientation_Z_rolling_mean',
       'AngularVelocity_X_rolling_mean', 'AngularVelocity_Y_rolling_mean',
       'AngularVelocity_Z_rolling_mean', 'BandAccX_rolling_mean',
       'BandAccY_rolling_mean', 'BandAccZ_rolling_mean', 'speed_rolling_mean',
       'rate_rolling_mean'],
      dtype='object', length=106)

In [4]:
# 获取数据集中所有常数列的列表
constant_columns = [col for col in X_train.columns if X_train[col].nunique() <= 1]

# 删除这些列
X_train = X_train.drop(columns=constant_columns)
X_test = X_test.drop(columns=constant_columns)

# 创建训练集的特征和目标变量
target = "act"  # 需要替换为你的目标列名
y_train = X_train[target]
X_train = X_train.drop(target, axis=1)

# 创建测试集的特征和目标变量
y_test = X_test[target]
X_test = X_test.drop(target, axis=1)

# 数据标准化
scaler = StandardScaler()

# 处理数据不平衡问题
smote_tomek = SMOTETomek(random_state=42)

# 使用XGBoost分类器
xgb_model = XGBClassifier()

# 创建流水线
pipeline = Pipeline(
    [
        ("scaler", scaler),
        ("smote_tomek", smote_tomek),
        (
            "feature_selection",
            RFE(estimator=xgb_model, n_features_to_select=10, step=1),
        ),
        ("classification", xgb_model),
    ]
)

# 定义超参数网格
param_grid = {
    "classification__n_estimators": [10, 50, 100],
    "classification__max_depth": [10, 20, 30],
    "classification__learning_rate": [0.01, 0.1, 0.2],
    "classification__subsample": [0.6, 0.8, 1.0],
    "classification__colsample_bytree": [0.6, 0.8, 1.0],
}

# 交叉验证
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 使用GridSearchCV进行超参数调优
grid_search = GridSearchCV(
    pipeline, param_grid, cv=cv, scoring="f1_macro", n_jobs=8, verbose=1
)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)
best_model = grid_search.best_estimator_

# 预测
y_pred = best_model.predict(X_test)

# 计算评估指标
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

precision = precision_score(y_test, y_pred, average="macro")
print(f"Precision: {precision}")

recall = recall_score(y_test, y_pred, average="macro")
print(f"Recall: {recall}")

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Root Mean Squared Error (RMSE): {rmse}")

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

f1 = f1_score(y_test, y_pred, average="macro")
print(f"F1 Score: {f1}")

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


RFE,XGB,无差分窗口特征：
Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best parameters found:  {'classification__colsample_bytree': 0.6, 'classification__learning_rate': 0.1, 'classification__max_depth': 20, 'classification__n_estimators': 200, 'classification__subsample': 1.0}
Accuracy: 0.24482352941176472
Precision: 0.27041683190551047
Recall: 0.3367577030812325
Mean Squared Error (MSE): 1.4625882352941177
Root Mean Squared Error (RMSE): 1.2093751424988517
Mean Absolute Error (MAE): 0.9682352941176471
F1 Score: 0.20595475913726044